In [ ]:
import numpy as np
import pycbc
from pycbc.frame import frame
import matplotlib.pyplot as plt

import sys
sys.path.append('/home/sreelakshmi.m/gwmat/src/')
import py_lgw
lgw = py_lgw.lensed_wf_gen()
#lgw = py_lgw.point_lens()

In [ ]:
import numpy as np
import pesummary
from pesummary.io import read
df_150914 = read("/home/sreelakshmi.m/downloads/IGWN-GWTC2p1-v2-GW150914_095045_PEDataRelease_mixed_cosmo.h5")
samples = df_150914.samples_dict
type(samples)
print(samples.keys())


In [ ]:
parameters_dict = samples["C01:IMRPhenomXPHM"]
pm_150914 = dict()
for key in parameters_dict.keys():
   pm_150914[key] = np.float64(np.median(parameters_dict[key]))

m1= pm_150914['mass_1']
m2= pm_150914['mass_2']
chi1= pm_150914['a_1']
chi2= pm_150914['a_2']
tilt1= pm_150914['tilt_1']
tilt2= pm_150914['tilt_2']
phi12= pm_150914['phi_12']
phijl= pm_150914['phi_jl']
thetajn= pm_150914['theta_jn']
dist = pm_150914['luminosity_distance']
ra = pm_150914['ra']
dec = pm_150914['dec']
pol = pm_150914['psi']
phase = pm_150914['phase']
time= round(pm_150914['geocent_time'])
rdf = lgw.f_RD_BBH(m1, m2, chi1, chi2, tilt1, tilt2, phi12)


In [ ]:
psd_pyr =np.loadtxt('/home/sreelakshmi.m/pyring/pyRing/data/PSDs/ASD_LIGO-P1200087-v18-aLIGO_DESIGN.txt')
freq = psd_pyr[: ,0]
ASD = psd_pyr[: ,1]
PSD = ASD**2
print(PSD)
file = '/home/sreelakshmi.m/pyring/pyRing/data/PSDs/PSD_LIGO-P1200087-v18-aLIGO_DESIGN.txt'
np.savetxt(file , np.column_stack((freq, PSD)))


In [ ]:
psd_V1_pyr = np.loadtxt('/home/sreelakshmi.m/pyring/pyRing/data/PSDs/ASD_LIGO-P1200087-v18-AdV_DESIGN.txt')
freq = psd_V1_pyr[: ,0]
ASD = psd_V1_pyr[: ,1]
PSD = ASD**2
file = '/home/sreelakshmi.m/pyring/pyRing/data/PSDs/PSD_LIGO-P1200087-v18-AdV_DESIGN.txt'
np.savetxt(file , np.column_stack((freq, PSD)))

In [ ]:
psd_file = '/home/sreelakshmi.m/pyring/pyRing/data/PSDs/PSD_LIGO-P1200087-v18-aLIGO_DESIGN.txt'
psd_file_V1 = '/home/sreelakshmi.m/pyring/pyRing/data/PSDs/PSD_LIGO-P1200087-v18-AdV_DESIGN.txt'

In [ ]:
import os
import itertools
import pickle
#folder_name = "sim_gw150914_ml"
#folder_path = os.path.join(os.getcwd() , folder_name)
folder_path = "/home/sreelakshmi.m/prod_runs/pyring/pyring_runs/s1_ul_vs_ml/data/set_0/gw150914/"


import itertools
Ml = [0, 1e2 , 1e3, 1e4]
y  = [0.1]
inp_lens_prms = list(itertools.product(y, Ml))

injection_info = dict()

for k, inp in enumerate(inp_lens_prms):
    y_val, Ml_val = inp
    init_prms = dict(f_start=20., f_low= rdf, f_high=None, f_ref=20., sample_rate=4096, wf_approximant="IMRPhenomXPHM", ifo_list=['H1', 'L1', 'V1'] )
    psd_prms = dict(Noise=False, psd_H1=psd_file, psd_L1=psd_file, psd_V1=psd_file_V1)
    lens_prms = dict(m_lens=Ml_val, y_lens=y_val, z_lens=0)  # (m_lens=0 means unlensed)
    cbc_prms = dict(mass_1=m1, mass_2=m2, a_1=chi1, a_2=chi2, tilt_1=tilt1, tilt_2=tilt2, 
            phi_jl=phijl, phi_12=phi12, theta_jn=thetajn, luminosity_distance= None, 
            ra=ra, dec=dec, polarization=pol, coa_phase=0, trig_time=time,  
            mode_array=None, lambda1=None, lambda2=None, rwrap = 0.)
    misc_prms = dict(extra_padding_at_start=5, extra_padding_at_end=5)
    prms = {**init_prms, **lens_prms, **cbc_prms, **psd_prms, **misc_prms} #'save_data':True, 'data_label':'simulated_data', 'data_outdir':'../src/'}

    dist = lgw.network_optimal_snr_to_distance(net_optimal_snr=30, **prms)
    print('required distance:', dist)
    prms['luminosity_distance'] = dist

    res = lgw.sim_lensed_noisy_wf_gen_with_snr(**prms)

    # findin actual trigger time at H1 (the reference detector)
    signal_H1 = res['pure_ifo_signal']['H1'] 
    max_index = np.argmax(np.array(signal_H1))
    peak_time = np.array(signal_H1.sample_times)[max_index]
    prms['trigger_time_at_H1'] = peak_time
    print('trigger_time_at_H1:', peak_time)
    injection_info[str(k)] = prms
    
    plt.plot(signal_H1.sample_times, signal_H1, color='red', linestyle='solid', label='WF: w/o noise (H1)')
    plt.xlim(1126259462 - 5, 1126259462+3)
    plt.legend()
    plt.show()
    
    ifos = ['H1', 'L1', 'V1']
   
    for ifo in ifos:
       channel_name = "PyCBC_Injection"
       signal = res['pure_ifo_signal'][ifo] 
       label = str(k) + "_" + ifo[0] + "-" + ifo + "_" + channel_name + "_" + str(round(prms["trig_time"])) + "-" + str(round(signal.duration)) 
       label += "_gw150914-like_Mlz_" + lgw.str_m(Ml_val) + "_yl_" + lgw.str_y(y_val)
       file_name = f"{label}"
       file_path = os.path.join(folder_path, file_name)
       #print(type(signal))
       signal.save(f"{file_path}.txt")



with open(folder_path + "injection_info.pkl", "wb") as f:
    pickle.dump(injection_info, f )        
# with open(folder_path + "injection_info.pkl", "rb") as f:
#     injection_info = pickle.load( f )   

In [ ]:
import numpy as np
import pesummary
from pesummary.io import read
df_190521 = read("/home/sreelakshmi.m/downloads/IGWN-GWTC2p1-v2-GW190521_030229_PEDataRelease_mixed_cosmo.h5")
samples_190521 = df_190521.samples_dict
type(samples_190521)
print(samples_190521.keys())

In [ ]:
parameters_190521 = samples_190521["C01:IMRPhenomXPHM"]
pm_190521 = dict()
for key in parameters_190521.keys():
   pm_190521[key] = np.float64(np.median(parameters_190521[key]))

m1_190521= pm_190521['mass_1']
m2_190521= pm_190521['mass_2']
chi1_190521= pm_190521['a_1']
chi2_190521= pm_190521['a_2']
tilt1_190521= pm_190521['tilt_1']
tilt2_190521= pm_190521['tilt_2']
phi12_190521= pm_190521['phi_12']
phijl_190521= pm_190521['phi_jl']
thetajn_190521= pm_190521['theta_jn']
dist_190521 = pm_190521['luminosity_distance']
ra_190521 = pm_190521['ra']
dec_190521 = pm_190521['dec']
pol_190521 = pm_190521['psi']
phase_190521 = pm_190521['phase']
time_190521= round(pm_190521['geocent_time'])
rdf_190521 = lgw.f_RD_BBH(m1_190521, m2_190521, chi1_190521, chi2_190521, tilt1_190521, tilt2_190521, phi12_190521)

In [ ]:
import os
import itertools
#folder_name = "sim_gw150914_ml"
#folder_path = os.path.join(os.getcwd() , folder_name)
folder_path = "/home/sreelakshmi.m/prod_runs/pyring/pyring_runs/s1_ul_vs_ml/data/set_0/gw190521A/"



import itertools
Ml = [0, 1e2 , 1e3, 1e4]
y  = [0.1]
inp_lens_prms = list(itertools.product(y, Ml))

injection_info = dict()

for k, inp in enumerate(inp_lens_prms):
    y_val, Ml_val = inp
    init_prms = dict(f_start=20., f_low= rdf_190521, f_high=None, f_ref=20., sample_rate=4096, wf_approximant="IMRPhenomXPHM", ifo_list=['H1', 'L1', 'V1'] )
    psd_prms = dict(Noise=False, psd_H1=psd_file, psd_L1=psd_file, psd_V1=psd_file_V1)
    lens_prms = dict(m_lens=Ml_val, y_lens=y_val, z_lens=0)  # (m_lens=0 means unlensed)
    cbc_prms = dict(mass_1=m1_190521, mass_2=m2_190521, a_1=chi1_190521, a_2=chi2_190521, tilt_1=tilt1_190521, tilt_2=tilt2_190521, 
                phi_jl=phijl_190521, phi_12=phi12_190521, theta_jn=thetajn_190521, luminosity_distance= None, 
                ra=ra_190521, dec=dec_190521, polarization=pol_190521, coa_phase=0, trig_time=time_190521,  
                mode_array=None, lambda1=None, lambda2=None, rwrap = -0.4)
    misc_prms = dict(extra_padding_at_start=10, extra_padding_at_end=10)
    prms = {**init_prms, **lens_prms, **cbc_prms, **psd_prms, **misc_prms} #'save_data':True, 'data_label':'simulated_data', 'data_outdir':'../src/'}
    
    dist = lgw.network_optimal_snr_to_distance(net_optimal_snr=30, **prms)
    print('required distance:', dist)
    prms['luminosity_distance'] = dist

    res = lgw.sim_lensed_noisy_wf_gen_with_snr(**prms)
    signal_H1 = res['pure_ifo_signal']['H1'] 
    max_index = np.argmax(np.array(signal_H1))
    peak_time = np.array(signal_H1.sample_times)[max_index]
    prms['trigger_time_at_H1'] = peak_time
    print('trigger_time_at_H1:', peak_time)
    injection_info[str(k)] = prms
    plt.plot(signal_H1.sample_times, signal_H1, color='red', linestyle='solid', label='WF: w/o noise (H1)')
    plt.xlim(1242442967- 5, 1242442967+3)
    plt.legend()
    plt.show()
    
    ifos = ['H1', 'L1', 'V1']
   
    for ifo in ifos:
       channel_name = "PyCBC_Injection"
       signal = res['pure_ifo_signal'][ifo] 
       label = str(k) + "_" + ifo[0] + "-" + ifo + "_" + channel_name + "_" + str(round(prms["trig_time"])) + "-" + str(round(signal.duration)) 
       label += "_gw190521-like_Mlz_" + lgw.str_m(Ml_val) + "_yl_" + lgw.str_y(y_val)
       file_name = f"{label}"
       file_path = os.path.join(folder_path, file_name)
       #print(type(signal))
       signal.save(f"{file_path}.txt")

with open(folder_path + "injection_info.pkl", "wb") as f:
    pickle.dump(injection_info, f )